In [2]:
%matplotlib inline

import matplotlib.pyplot as plt
import pandas as pd
import warnings
from sklearn.model_selection import train_test_split

from utils import make_dataset

warnings.filterwarnings('ignore')

data = pd.read_csv('../data/processed/compas-scores-two-years-processed.csv')

DROP_COLS = ['two_year_recid','compas_score','decile_score','compas_class']
FEATURE_COLS = data.drop(DROP_COLS, axis=1).columns.tolist()

train, test = train_test_split(data, test_size=0.2, random_state=1234)

X_train, y_train = train[FEATURE_COLS], train['two_year_recid']
X_test, y_test = test[FEATURE_COLS], test['two_year_recid']

In [3]:
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=1234)
clf.fit(X_train, y_train)

y_test_pred = clf.predict_proba(X_test)
test['recid_prediction_score'] = y_test_pred[:,1]
test['recid_prediction_class'] = (test['recid_prediction_score'] >0.5).astype(int)

acc = accuracy_score(y_test, test['recid_prediction_class'])

In [4]:
from aif360.metrics.classification_metric import ClassificationMetric
from aif360.datasets import BinaryLabelDataset

BIAS_INFO = {'favorable_label':0,
             'unfavorable_label':1,
             'protected_columns':['race']
            }
PRIVILEGED_INFO = {'unprivileged_groups':[{'race': 2},
                                          {'race': 1},
                                          {'race': 4},
                                          {'race': 5},
                                          {'race': 6}],
                   'privileged_groups':[{'race': 3}]
                  }

ground_truth_test = make_dataset(test[FEATURE_COLS], test['two_year_recid'], **BIAS_INFO, **PRIVILEGED_INFO)
prediction_test = make_dataset(test[FEATURE_COLS], test['recid_prediction_class'], test['recid_prediction_score'], **BIAS_INFO, **PRIVILEGED_INFO)

clf_metric = ClassificationMetric(ground_truth_test, prediction_test,**PRIVILEGED_INFO)

In [5]:
import joblib
from utils import display_results

joblib.dump((clf_metric,acc), '../results/0.2-baseline_model-race.pkl')
display_results('../results/0.2-baseline_model-race.pkl')

,metric_names,scores
0,accuracy_score,0.671518
1,true_positive_rate_difference,-0.102441
2,false_positive_rate_difference,-0.247320
3,false_omission_rate_difference,-0.035370
4,false_discovery_rate_difference,-0.042279
5,error_rate_difference,-0.034496
6,false_positive_rate_ratio,0.621169
7,false_negative_rate_ratio,1.714525
8,false_omission_rate_ratio,0.905385
9,false_discovery_rate_ratio,0.877524


In [6]:
display_results('../results/0.1-compas_model-race.pkl')

,metric_names,scores
0,accuracy_score,0.652114
1,true_positive_rate_difference,-0.102481
2,false_positive_rate_difference,-0.177586
3,false_omission_rate_difference,0.019860
4,false_discovery_rate_difference,-0.002715
5,error_rate_difference,0.000637
6,false_positive_rate_ratio,0.749823
7,false_negative_rate_ratio,2.058970
8,false_omission_rate_ratio,1.061051
9,false_discovery_rate_ratio,0.992292


## Questions

* Is our baseline model better than compas model?
* Is it one model better on all metrics?

# Run this analysis for column 'sex'

## Questions

**Is our model more race-biased or sex-biased?**

**On which metrics that is differ the most?**